In [ ]:
import sys
import pycochleagram.cochleagram as cgram
from pycochleagram import utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import sys
import os
from os import listdir

from sklearn.model_selection import KFold
from sklearn.linear_model import ElasticNetCV, LinearRegression, RidgeCV
from sklearn.metrics import r2_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms

from forest_sound_dataset import SoundDataset,SoundDataset_kfold
from PredNet_architectures_9_1_2024 import PredNet_test
from rnn_model_functions import *

This notebook illustrates how to train the Gaussian Linear model as well as to get the prediction correlation scores

In [ ]:
full_sound_input_normalized = np.load("/Users/ariellerosinski/My Drive/UCL/MSc/Project/Code/full_coch/full_pennington_david.npy")/1000

In [ ]:
#Get Hidden Unit Responses
full_sound_input_tensor = torch.tensor(full_sound_input_normalized, requires_grad=False) 
full_sound_input_tensor = full_sound_input_tensor[None,None,:,:]

batch_sz=1
hidden_size=128
time_lag=30 
burn_in=30 

device = torch.device('cpu')
model = PredNet_test(time_lag=time_lag, burn_in=burn_in, hidden_size=hidden_size).float()
model_path="/path to .pt model"
model.load_state_dict(torch.load(model_path, map_location=device))
x=(model.encoder(full_sound_input_tensor.float(),initialization=None)[0]).detach().numpy()    #x_shape = (1, 44475, 128)
x_comp = x[:,:-25,:]
x_comp = np.transpose(x_comp.squeeze())                                                       #(128, 44450)


In [ ]:
#Get A1 Data
resp_A1 = np.load("/Users/ariellerosinski/My Drive/UCL/MSc/Project/Code/Pennington_David_Analyses/respA1.npy")   #(849, 88950)

def get_resp(resp):
    resp = resp[:,25:-25]
    resp = resp.reshape(resp.shape[0], -1, 2)
    resp = np.mean(resp, axis=2)
    return resp

In [ ]:
#RIDGE REGRESSION:
resp = get_resp(resp_A1)                                #(849, 44450)
x = x_comp 
regr = RidgeCV(alphas=(0.1, 1.0, 10.0, 1e2)) 
reg_est = regr.fit(x.T[1350:,:], resp.T[1350:,:])     
print(reg_est.score(x.T[:1325,:], resp.T[:1325,:]))
print(regr.alpha_)

y_pred_val = reg_est.predict(x.T[:1325,:])
print(y_pred_val.shape)
r_2_est = reg_est.score(x.T[:1325,:], resp.T[:1325,:])

#for an example cell
print(r2_score(resp.T[:1325,233], y_pred_val[:,233]))     

In [ ]:
#Example cell predictions alongside true PSTHs
full_sound_input_normalized_reshaped = full_sound_input_normalized[:,:-25]
plt.figure()
pred = y_pred_val
r_2_value = r_2_est

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(20, 5)) 
img = ax1.imshow(full_sound_input_normalized_reshaped[:, :600], aspect='auto', origin='lower', interpolation='none', cmap='gray_r')

ax1.set_title('Input cochleagram')
ax2.plot(pred[:600,233],label="prediction", color='forestgreen', linestyle='dashed' )#alpha=0.5
ax2.plot(resp.T[:600,233],label="actual", color='cornflowerblue')
ax2.set_title(f'r2={r2_score(resp.T[:1325,233], y_pred_val[:,233])}')

ax3.plot(pred[:600,846],label="prediction",  color='forestgreen', linestyle='dashed')       #846
ax3.plot(resp.T[:600,846],label="actual",  color='cornflowerblue') 
ax3.set_title(f'r2={r2_score(resp.T[:1325,846], y_pred_val[:,846])}')

plt.legend()
plt.suptitle(f'r2={r_2_value}')

PREDICTION CORRELATION

In [ ]:
raster_A1_idx = [135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 678, 679, 680, 681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 692, 693, 694, 695, 696, 697, 698, 699, 700, 701, 702, 703, 704, 705, 706, 707, 708, 709, 710, 711, 712, 713, 714, 715, 716, 717, 718, 719, 720, 721, 722, 723, 724, 725, 726, 727, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605, 606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 728, 729, 730, 731, 732, 733, 734, 735, 736, 737, 738, 739, 740, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641, 642, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 741, 742, 743, 744, 745, 746, 747, 748, 749, 750, 751, 752, 753, 754, 755, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, None, None, None, None, None, None, None, None, None, None, None, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 756, 757, 758, 759, 760, 761, 762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792, 793, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 804, 805, 806, 807, 808, 809, 810, 811, 812, 813, 814, 815, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]

In [ ]:
def get_spikes_val_single_pres(file, cell_id_raster):
    spikes_val = np.load(file)                                             #(20, 816, 26500)
    spikes_val = spikes_val[:,cell_id_raster,np.newaxis,:]                 #(20, 1, 26500)
    spikes_val = spikes_val.reshape(spikes_val.shape[0], -1, 20)
    spikes_val = np.sum(spikes_val, axis=2)                                #(20, 1325)
    return spikes_val

In [ ]:
ls_pred_corr =[]
ls_small_ttrc = []
for i in range(y_pred_val.shape[1]):
    cell_id = i
    cell_id_raster = raster_A1_idx[cell_id] #373
    print("neuron", i)

    if cell_id_raster is not None:
        y_true_val = resp.T[:1325,cell_id]

        y_pred_val_cell_i = y_pred_val[:,cell_id]

        correlation = np.corrcoef(y_true_val[:], y_pred_val_cell_i[:])[0, 1]
    
    
        spikes_val_single_pres = get_spikes_val_single_pres("/Users/ariellerosinski/My Drive/UCL/MSc/Project/Code/Pennington_David_Analyses/raster_cells_val.npy", cell_id_raster=cell_id_raster )

        ls_corr_single_pres = []
        for i in range(spikes_val_single_pres.shape[0]):
            ls_corr_single_pres.append(np.corrcoef(spikes_val_single_pres[i,:], y_pred_val_cell_i[:])[0,1])
        
        corr_single_pres_avg = np.nanmean(ls_corr_single_pres)

        correlation_coefficients = np.corrcoef(spikes_val_single_pres)
        correlation_coefficients = np.ma.masked_array(correlation_coefficients, np.eye(correlation_coefficients.shape[0], dtype=bool))      #exclude diag values i.e., correlation of 1
        ttrc = np.nanmean(correlation_coefficients)                                                                                            #mean_correlation
       
        if ttrc > 0.02:
            ls_pred_corr.append(corr_single_pres_avg/np.sqrt(ttrc))
        else:
            ls_pred_corr.append(correlation)
            ls_small_ttrc.append(i)
    else:
        ls_pred_corr.append(np.nan)
        print(None)

np.save("/save path for Gaussian LN pred corr", np.array(ls_pred_corr))

In [ ]:
pred_corr = np.load("/save path for Gaussian LN pred corr")
np.nanmedian(pred_corr)

Similar for PC-based model except for small changes:
- resp_A1 = resp_A1_avg[:,(resp_A1_avg.shape[1] - pc_pred.shape[1]):]
- est [:,1321:] and val [:,:1296] indices
- spikes_val_single_pres = spikes_val_single_pres[:,29:]